In [ ]:
!pip install dash --upgrade
!pip install plotly --upgrade

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd

In [90]:
app = dash.Dash(__name__)


In [ ]:
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv"

# Use pandas to read the CSV file and store it in a DataFrame
spacex_df = pd.read_csv(url)

In [ ]:
options = [
    {'label': 'All Sites', 'value': 'ALL'},
    {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
    {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
    {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
    {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
]

In [91]:
app.layout = html.Div([
    html.H1("SPACEX LAUNCH RECORDS DASHBOARD", style = {'text-align': 'center'}),
    dcc.Dropdown(
        id='dropdown-input',
        options=options,
        value='ALL',
        placeholder="CCAFS SLC-40",  # Set your placeholder here
        searchable=True
    ),
    dcc.Graph(id="success-pie-chart"),
    dcc.Graph(id="payload-scatter-plot"),
     html.Div([
        html.Label("Payload Range Slider", style={'font-weight': 'bold'}),
        dcc.RangeSlider(id='payload-slider',
                min=0, max=10000, step=1000,
                value=[0, 10000])
     ])
    # Add other components as needed
])


In [92]:
# Function decorator to specify function input and output
@app.callback([Output(component_id='success-pie-chart', component_property='figure'),
              Output(component_id='payload-scatter-plot', component_property='figure')],
              [Input(component_id='dropdown-input', component_property='value'),Input('payload-slider', 'value')])
def get_pie_chart(entered_site, payload_range):
    if entered_site == 'ALL':
        filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= payload_range[0]) & 
                            (spacex_df['Payload Mass (kg)'] <= payload_range[1])]
        fig = px.pie(filtered_df, values='class', 
        names='Launch Site', 
        title='Success Rate for all Launch Sites')
    else:
        # return the outcomes piechart for a selected site
        filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= payload_range[0]) & 
                            (spacex_df['Payload Mass (kg)'] <= payload_range[1]) &
                                (spacex_df['Launch Site'] == entered_site)]
        success_count = filtered_df['class'].sum()
        failure_count = len(filtered_df) - success_count
        count = [success_count, failure_count]
        fig = px.pie(filtered_df, values=count, 
        names=[1,0], 
        title=f'Success Rate for {entered_site}')
        
    if entered_site == 'ALL':
        # If ALL sites are selected, render a scatter plot with all data
        scatter_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= payload_range[0]) & 
                            (spacex_df['Payload Mass (kg)'] <= payload_range[1])]
        title = 'Payload Mass vs. Class (All Sites)'
        scatter_fig = px.scatter(
            scatter_df,
            x='Payload Mass (kg)',
            y='class',
            color='Booster Version',
            title=title
        )
    else:
        # If a specific launch site is selected, filter spacex_df
        scatter_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= payload_range[0]) & 
                            (spacex_df['Payload Mass (kg)'] <= payload_range[1]) &
                               (spacex_df['Launch Site'] == entered_site)]
        title = f'Payload Mass vs. Class ({entered_site})'
        scatter_fig = px.scatter(
            scatter_df,
            x='Payload Mass (kg)',
            y='class',
            color='Booster Version',
            title=title
        )

    return fig, scatter_fig

In [102]:
if __name__ == '__main__':
    app.run_server(mode = "inline",debug=True)